In [ ]:
# Libraries
# hide
import pandas as pd
import numpy as np
from pprint import pprint
from gensim.models.doc2vec import Doc2Vec
from ds4se.mgmnt.prep.conv import *

In [ ]:
!pwd

/tf/main/nbs


## Data load

In [ ]:
java_df = pd.read_csv("test_data/java/clean_java.csv")

In [ ]:
java_df.shape

(496494, 11)

In [ ]:
with open("test_data/java-gen.jsonl") as f:
    content = f.readlines()
len(content)

454451

In [ ]:
len(content)

454451

In [ ]:
content[0][1:-2]

'"Optional",",","\\"\\"","int","r","!=","8","String","fieldName","<","Object",")","new","Date","endTime","}","}","break","charloop","e",")","+","\\", \\"","String","password",";","List","properties","+","key","^=","m_pbox","for",":","c","if","get","?","value","&&","null","(","new","java",":","String","languageStr","\\"\\"","null",">","T","call","null","]","value",":","valueSet",";","long","r","(","i","&","0x000000ff",")",">","Level","l","|=","(","i","{","out",",","encoding","null","^","typeArgResult","return","new","Matcher","groupMatcher","return","map","String","uri","!=","0.0",">","response","<","2","public","final","int","getBeUint24","<","Boolean","[","ids","public","synchronized","XAttribute","+","key","(","getWp","this","]","bytes","/=","size","->",")","[","index","[","destIdx","+","\\" \\"","!=","value","||","t","[","9","||","null","int","end","?","Double","total","|=","add","]","&","0xffL","&","PROPERTY_IDENTITY","throw","e",";","size","]","]","[",")","return","value","-","5",

In [ ]:
import re
import ast

In [ ]:
list(ast.literal_eval(content[0][1:-2]))

['Optional',
 ',',
 '""',
 'int',
 'r',
 '!=',
 '8',
 'String',
 'fieldName',
 '<',
 'Object',
 ')',
 'new',
 'Date',
 'endTime',
 '}',
 '}',
 'break',
 'charloop',
 'e',
 ')',
 '+',
 '", "',
 'String',
 'password',
 ';',
 'List',
 'properties',
 '+',
 'key',
 '^=',
 'm_pbox',
 'for',
 ':',
 'c',
 'if',
 'get',
 '?',
 'value',
 '&&',
 'null',
 '(',
 'new',
 'java',
 ':',
 'String',
 'languageStr',
 '""',
 'null',
 '>',
 'T',
 'call',
 'null',
 ']',
 'value',
 ':',
 'valueSet',
 ';',
 'long',
 'r',
 '(',
 'i',
 '&',
 '0x000000ff',
 ')',
 '>',
 'Level',
 'l',
 '|=',
 '(',
 'i',
 '{',
 'out',
 ',',
 'encoding',
 'null',
 '^',
 'typeArgResult',
 'return',
 'new',
 'Matcher',
 'groupMatcher',
 'return',
 'map',
 'String',
 'uri',
 '!=',
 '0.0',
 '>',
 'response',
 '<',
 '2',
 'public',
 'final',
 'int',
 'getBeUint24',
 '<',
 'Boolean',
 '[',
 'ids',
 'public',
 'synchronized',
 'XAttribute',
 '+',
 'key',
 '(',
 'getWp',
 'this',
 ']',
 'bytes',
 '/=',
 'size',
 '->',
 ')',
 '[',
 'index',

In [ ]:
print("asdfasdf")

asdfasdf


In [ ]:
def get_df_from_ngram_gen(gens):
    result_df = pd.DataFrame([])
    splitted = []
    for gen in gens:
        splitted.append(list(ast.literal_eval(gen[1:-2])))
    result_df["splitted_code"] = splitted
    return result_df

In [ ]:
ngram_df = get_df_from_ngram_gen(content)

In [ ]:
ngram_df.shape

(454451, 1)

In [ ]:
ngram_df.head()

,splitted_code
0,"[Optional, ,, """", int, r, !=, 8, String, field..."
1,"[void, preProcessOperation, int, size, --, !=,..."
2,"[Expr, and, i, /, 5, -, 1L, ,, new, Color, col..."
3,"[static, Function, javaMethod, ), :, Arrays, !..."
4,"[Type, scanTimestamp, final, Type, childType, ..."


In [ ]:
ngram_df_sample = ngram_df.sample(10000, random_state=2)

In [ ]:
ngram_df_sample.shape

(10000, 1)

In [ ]:
df_train = java_df[java_df["partition"] == "train"]
df_valid = java_df[java_df["partition"] == "valid"]
df_test = java_df[java_df["partition"] == "test"]

In [ ]:
df_train.head()

,Unnamed: 0,code,code_len,code_tokens,cyclomatic_complexity,data_type,method_name,nloc,parameter_count,partition,token_count
26900,0,protected final void bindIndexed(Configuration...,80.0,"['protected', 'final', 'void', 'bindIndexed', ...",4.0,src,bindIndexed,11.0,6.0,train,75.0
26901,1,public void setServletRegistrationBeans(\n\t\t...,37.0,"['public', 'void', 'setServletRegistrationBean...",1.0,src,setServletRegistrationBeans,6.0,1.0,train,34.0
26902,2,public void addServletRegistrationBeans(\n\t\t...,33.0,"['public', 'void', 'addServletRegistrationBean...",1.0,src,addServletRegistrationBeans,6.0,1.0,train,29.0
26903,3,public void setServletNames(Collection<String>...,32.0,"['public', 'void', 'setServletNames', '(', 'Co...",1.0,src,setServletNames,4.0,1.0,train,31.0
26904,4,public void addServletNames(String... servletN...,33.0,"['public', 'void', 'addServletNames', '(', 'St...",1.0,src,addServletNames,4.0,1.0,train,31.0


In [ ]:
print(f"df_train shape: {df_train.shape}")
print(f"df_valid shape: {df_valid.shape}")
print(f"df_test shape: {df_test.shape}")

df_train shape: (454273, 11)
df_valid shape: (15321, 11)
df_test shape: (26900, 11)


In [ ]:
doc2_vec_model_path = "test_data/models/pv/[doc2vec-Py-Java-Wiki-PVDBOW-500-20E[15]-1592941134.367976].model"

In [ ]:
model = Doc2Vec.load(doc2_vec_model_path)

In [ ]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [ ]:
vector = model.infer_vector(splitted_code["splitted_code"][0])

In [ ]:
vector.shape

(500,)

In [ ]:
splitted_code["splitted_code"][0]

['public',
 'final',
 'option',
 'find',
 'first',
 'queri',
 'string',
 'queri',
 'string',
 'sort',
 'object',
 'param',
 'find',
 'find',
 'collect',
 'find',
 'queri',
 'param',
 'sort',
 'null',
 'sort',
 'empti',
 'find',
 'sort',
 'sort',
 'iter',
 'find',
 'limit',
 'entiti',
 'class',
 'iter',
 'iter',
 'iter',
 'iter',
 'next',
 'return',
 'option',
 'iter',
 'next',
 'return',
 'option',
 'empti']

In [ ]:
processing_params = {
    "system": "codesearchnet",
    "saving_path": "test_data/java/",
    "language": "english",
}

In [ ]:
cp = ConventionalPreprocessing(processing_params)

In [ ]:
sample_java_df = df_train.sample(10000, random_state=2)

In [ ]:
sample_java_df_test = df_test.sample(10000, random_state=2)

In [ ]:
processed_df = cp.fromdocs_pipeline(sample_java_df["code"])

2020-07-06 02:08:19,774 : INFO : fromtokens_pipeline: clean punctuation
2020-07-06 02:08:20,470 : INFO : fromtokens_pipeline: camel case
2020-07-06 02:08:20,486 : INFO : fromtokens_pipeline: lowe case
2020-07-06 02:08:20,503 : INFO : fromtokens_pipeline: white space removed
2020-07-06 02:08:21,047 : INFO : fromtokens_pipeline: WordPunctTokenizer
2020-07-06 02:08:24,892 : INFO : fromtokens_pipeline: Stop words
2020-07-06 02:08:37,037 : INFO : fromtokens_pipeline: Stemmings
2020-07-06 02:08:37,317 : INFO : fromtokens_pipeline: Removed Special Terns
2020-07-06 02:08:37,344 : INFO : fromtokens_pipeline END


In [ ]:
processed_test_df = cp.fromdocs_pipeline(sample_java_df_test["code"])

2020-07-06 22:05:44,086 : INFO : fromtokens_pipeline: clean punctuation
2020-07-06 22:05:44,589 : INFO : fromtokens_pipeline: camel case
2020-07-06 22:05:44,597 : INFO : fromtokens_pipeline: lowe case
2020-07-06 22:05:44,606 : INFO : fromtokens_pipeline: white space removed
2020-07-06 22:05:45,008 : INFO : fromtokens_pipeline: WordPunctTokenizer
2020-07-06 22:05:47,985 : INFO : fromtokens_pipeline: Stop words
2020-07-06 22:05:57,564 : INFO : fromtokens_pipeline: Stemmings
2020-07-06 22:05:57,761 : INFO : fromtokens_pipeline: Removed Special Terns
2020-07-06 22:05:57,775 : INFO : fromtokens_pipeline END


In [ ]:
processed_df[0]

'public final option find first queri string queri string sort object param find find collect find queri param sort null sort empti find sort sort iter find limit entiti class iter iter iter iter next return option iter next return option empti'

In [ ]:
def get_df_from_processed_list(processed_list):
    result_df = pd.DataFrame([])
    p_list = []
    for element in processed_list:
        p_list.append(element.split())
    result_df["splitted_code"] = p_list
    return result_df

In [ ]:
splitted_code = get_df_from_processed_list(processed_df)

In [ ]:
splitted_test_code = get_df_from_processed_list(processed_test_df)

In [ ]:
splitted_code["splitted_code"][0]

['public',
 'final',
 'option',
 'find',
 'first',
 'queri',
 'string',
 'queri',
 'string',
 'sort',
 'object',
 'param',
 'find',
 'find',
 'collect',
 'find',
 'queri',
 'param',
 'sort',
 'null',
 'sort',
 'empti',
 'find',
 'sort',
 'sort',
 'iter',
 'find',
 'limit',
 'entiti',
 'class',
 'iter',
 'iter',
 'iter',
 'iter',
 'next',
 'return',
 'option',
 'iter',
 'next',
 'return',
 'option',
 'empti']

In [ ]:
def compute_vectors(code_df):
    vectors = []
    for idx, row in code_df.iterrows():
        vectors.append(model.infer_vector(row["splitted_code"]))
    return vectors

In [ ]:
doc2_vectors = compute_vectors(splitted_code)

In [ ]:
doc2_vectors = np.array(doc2_vectors)

In [ ]:
pprint(doc2_vectors[0:2])

[array([-0.37762475, -0.38540062, -0.01277232,  0.04222575,  0.34180853,
        0.11128339, -0.87900287, -0.12815368,  0.12353008,  0.12468721,
        0.21387072, -0.6470642 ,  0.05903836,  0.2746196 ,  0.19446734,
        0.24344777,  0.5203615 , -0.30637196, -0.18092875,  0.21426082,
       -0.1562226 , -0.38738626,  0.15108353, -0.3845797 , -0.10619601,
       -0.25258663,  0.37857908, -0.5802679 , -0.23103325, -0.08668967,
        0.45152465,  0.03575516,  0.01235824, -0.3029284 , -0.2672066 ,
       -0.16894214, -0.27964777,  0.27612564, -0.3302533 ,  0.5936772 ,
        0.70220333, -0.06169656, -0.06106625, -0.12760344, -0.6531744 ,
       -0.00669117, -0.06631754, -0.48467466, -0.41192317,  0.3416588 ,
        0.41475022,  0.2056126 ,  0.21884891,  0.46283868,  0.684039  ,
        0.10750329, -0.02496543, -0.12638465, -0.3135548 ,  0.47960237,
        0.24430148, -0.39946005,  0.24253428, -0.11794276, -0.24581376,
       -0.15839438,  0.055303  , -0.2543045 , -0.05506008,  0.4

In [ ]:
np.save("test_data/java/doc2_vec_sample_java_df", doc2_vectors)

In [ ]:
ngram_vectors = compute_vectors(ngram_df_sample)

In [ ]:
len(ngram_vectors)

10000

In [ ]:
np.save("test_data/java/doc2_vec_ngram_df", ngram_vectors)

In [ ]:
java_test_vectors = compute_vectors(splitted_test_code)

In [ ]:
len(java_test_vectors)

10000

In [ ]:
np.save("test_data/java/doc2_vec_java_test_df", java_test_vectors)

In [ ]:
sample_java_df.to_csv("test_data/java/sample_java_df.csv")

In [ ]:
sample_java_df_test.to_csv("test_data/java/sample_java_test_df.csv")

In [ ]:
ngram_df_sample.to_csv("test_data/java/ngram_sample_df.csv")